In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
dfBalanceSheet = pd.read_pickle("BalanceSheet_20211228_165234.pkl")
dfCashflow = pd.read_pickle("CashFlow_20211228_165234.pkl")
# dfEarnings = pd.read_pickle("Earnings_20211228_165234.pkl")
dfFinancials = pd.read_pickle("Financials_20211228_165234.pkl")
dfInfo = pd.read_pickle("pkl/Info_0.pkl")

In [3]:
# dfBalanceSheet = dfBalanceSheet[["Date","Symbol","Price","Common Stock","Total Stockholder Equity"]]
# dfCashflow = dfCashflow[["Date","Symbol","Dividends Paid"]]
# dfFinancials = dfFinancials[["Date","Symbol","Total Revenue","Net Income"]]
# dfInfo = dfInfo[["symbol", "sector", "industry", "isEsgPopulated", "beta", "revenueGrowth", "earningsGrowth", "recommendationKey", "targetLowPrice", "targetMeanPrice", "targetMedianPrice", "targetHighPrice"]]

In [4]:
# dfBalanceSheet.drop(columns=["Intangible Assets","Capital Surplus","Total Liab","Minority Interest","Other Current Liab","Total Assets",
#                             "Other Current Assets","Retained Earnings","Other Liab","Good Will","Treasury Stock","Other Assets","Cash","Total Current Liabilities",
#                             "Deferred Long Term Asset Charges","Other Stockholder Equity","Property Plant Equipment","Total Current Assets","Net Tangible Assets",
#                             "Net Receivables","Inventory","Accounts Payable","Deferred Long Term Liab","Short Long Term Debt","Long Term Investments","Short Term Investments","Long Term Debt"], inplace=True, index=None)

# dfCashflow.drop(columns=["Net Income","Change To Liabilities","Total Cashflows From Investing Activities","Net Borrowings","Total Cash From Financing Activities","Change To Operating Activities",
#                         "Change In Cash","Effect Of Exchange Rate","Total Cash From Operating Activities","Depreciation","Other Cashflows From Investing Activities","Change To Inventory",
#                         "Change To Account Receivables","Change To Netincome","Capital Expenditures","Issuance Of Stock","Other Cashflows From Financing Activities","Investments","Repurchase Of Stock"],inplace=True,index=None)

# dfFinancials.drop(columns=["Research Development","Effect Of Accounting Charges","Income Before Tax","Minority Interest","Selling General Administrative",
#                           "Gross Profit","Ebit","Operating Income","Other Operating Expenses","Interest Expense","Extraordinary Items","Non Recurring",
#                           "Other Items","Income Tax Expense","Total Operating Expenses","Cost Of Revenue","Total Other Income Expense Net",
#                            "Discontinued Operations","Net Income From Continuing Ops","Net Income Applicable To Common Shares"],inplace=True,index=None)

# dfBalanceSheet["Price"].replace("N/A",np.nan,inplace=True)

In [5]:
dfBalanceSheet["Date"] = pd.to_datetime(dfBalanceSheet.index)
dfBalanceSheet.sort_values(by=["Symbol","Date"],axis=0, inplace=True)
dfBalanceSheet.reset_index(drop=True, inplace=True)

dfCashflow["Date"] = pd.to_datetime(dfCashflow.index)
dfCashflow.sort_values(by=["Symbol","Date"],axis=0, inplace=True)
dfCashflow.reset_index(drop=True, inplace=True)

dfFinancials["Date"] = pd.to_datetime(dfFinancials.index)
dfFinancials.sort_values(by=["Symbol","Date"],axis=0, inplace=True)
dfFinancials.reset_index(drop=True, inplace=True)

In [6]:
dfBalanceSheet = dfBalanceSheet[["Date","Symbol","Price","Common Stock","Total Stockholder Equity"]]
dfCashflow = dfCashflow[["Date","Symbol","Dividends Paid"]]
dfFinancials = dfFinancials[["Date","Symbol","Total Revenue","Net Income"]]
dfInfo = dfInfo[["symbol", "sector", "industry", "isEsgPopulated", "beta", "revenueGrowth", "earningsGrowth", "recommendationKey", "targetLowPrice", "targetMeanPrice", "targetMedianPrice", "targetHighPrice", "currentPrice", "sharesOutstanding", "totalRevenue", "bookValue", "dividendRate", "forwardEps", "payoutRatio", "forwardPE", "returnOnEquity"]]
dfInfo["PBV Ratio"] = dfInfo["currentPrice"]/dfInfo["bookValue"]
dfInfo["symbol"] = dfInfo["symbol"].str.split(".",0).str[0]

In [9]:
dfInfo[dfInfo["isEsgPopulated"] == True]

,symbol,sector,industry,isEsgPopulated,beta,revenueGrowth,earningsGrowth,recommendationKey,targetLowPrice,targetMeanPrice,...,currentPrice,sharesOutstanding,totalRevenue,bookValue,dividendRate,forwardEps,payoutRatio,forwardPE,returnOnEquity,PBV Ratio
15,ADVANC,Communication Services,Telecom Services,True,0.084955,0.016,-0.023,buy,180,233.39,...,219,2974000128,177068605440,25.124,6.9,9.63,0.7788,22.860125,0.37953,8.716765
46,AOT,Industrials,Airports & Air Services,True,0.823008,-0.175,None,buy,45,69.03,...,60.75,14285700096,7303883264,7.864,None,1.08,None,56.249996,-0.12768,7.725076


In [46]:
merged_df = dfBalanceSheet.merge(dfCashflow, how = "inner", on = ["Symbol", "Date"])
merged_df = merged_df.merge(dfFinancials, how = "inner", on = ["Symbol", "Date"])

In [47]:
merged_df["Book Value per Share"] = merged_df["Total Stockholder Equity"]/merged_df["Common Stock"]
merged_df["Dividend"] = abs(merged_df["Dividends Paid"]/merged_df["Common Stock"])
merged_df["Earning per Share"] = merged_df["Net Income"]/merged_df["Common Stock"]
merged_df["Payout Ratio"] = merged_df["Dividend"]/merged_df["Earning per Share"]
merged_df["PE Ratio"] = merged_df["Price"]/merged_df["Earning per Share"]
merged_df["PBV Ratio"] = merged_df["Price"]/merged_df["Book Value per Share"]
merged_df["ROE"] = merged_df["Net Income"]/merged_df["Total Stockholder Equity"]
# merged_df["ROE"] = merged_df["Net Income"]/(merged_df["Common Stock"])
merged_df

,Date,Symbol,Price,Common Stock,Total Stockholder Equity,Dividends Paid,Total Revenue,Net Income,Book Value per Share,Dividend,Earning per Share,Payout Ratio,PE Ratio,PBV Ratio,ROE
0,2017-12-31,2S,4.00,399999000.0,1281838000.0,-54000000.0,4523720000.0,284707000.0,3.204603,0.135,0.711769,0.189669,5.619799,1.248205,0.222108
1,2018-12-31,2S,2.07,449999000.0,1315885000.0,-71500000.0,5449510000.0,105547000.0,2.924195,0.158889,0.234549,0.677423,8.825432,0.707887,0.08021
2,2019-12-31,2S,1.80,449999000.0,1379550000.0,-63000000.0,5396274000.0,133382000.0,3.065673,0.14,0.296405,0.472328,6.07277,0.587147,0.096685
3,2020-12-31,2S,3.78,449999000.0,1757483000.0,-31500000.0,5603025000.0,410165000.0,3.905526,0.07,0.91148,0.076798,4.147102,0.967859,0.233382
4,2017-12-31,3K-BAT,NaN,200000000.0,1569203796.0,-60000000.0,5086840736.0,-348726881.0,7.846019,0.3,-1.743634,-0.172054,NaN,NaN,-0.222232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3308,2020-12-31,ZEN,10.90,300000000.0,1211125000.0,-134992000.0,2333390000.0,-63655000.0,4.037083,0.449973,-0.212183,-2.120682,-51.370668,2.699969,-0.052559
3309,2017-12-31,ZIGA,4.96,260000000.0,782249632.0,-120004000.0,882234966.0,131675688.0,3.008652,0.461554,0.506445,0.91136,9.793759,1.648579,0.168329
3310,2018-12-31,ZIGA,1.76,260000000.0,757214098.0,-67600000.0,841144191.0,42564466.0,2.912362,0.26,0.163709,1.588179,10.750752,0.60432,0.056212
3311,2019-12-31,ZIGA,1.02,260000000.0,752809446.0,-41600000.0,760165583.0,35260109.0,2.895421,0.16,0.135616,1.179803,7.521247,0.35228,0.046838


In [8]:
# dfEarnings.drop(columns=["Open","High","Low","Close","Adj Close","Volume"])

In [48]:
merged_df.loc[merged_df["Symbol"] == "ADVANC"]

,Date,Symbol,Price,Common Stock,Total Stockholder Equity,Dividends Paid,Total Revenue,Net Income,Book Value per Share,Dividend,Earning per Share,Payout Ratio,PE Ratio,PBV Ratio,ROE
56,2017-12-31,ADVANC,191.0,2973095330.0,50320354653.0,-2.319009e+10,157721800352.0,30077312101.0,16.925241,7.799981,10.116498,0.771016,18.880052,11.284921,0.597717
57,2018-12-31,ADVANC,172.5,2973095330.0,57521039809.0,-2.185181e+10,169855847157.0,29682178137.0,19.34719,7.349853,9.983594,0.736193,17.278346,8.916024,0.516023
58,2019-12-31,ADVANC,213.0,2973178632.0,69266099106.0,-2.104876e+10,180893685238.0,31189570687.0,23.296985,7.079548,10.490312,0.674865,20.304449,9.142814,0.450286
59,2020-12-31,ADVANC,176.0,2973554313.0,75563983962.0,-2.021782e+10,172890265287.0,27434360343.0,25.412007,6.799211,9.226117,0.736953,19.076281,6.92586,0.363061


In [28]:
tickers = merged_df["Symbol"].drop_duplicates(keep="first", inplace=False)
tickers.reset_index(drop=True, inplace=True)
tickers

0          2S
1      3K-BAT
2         7UP
3           A
4          A5
        ...  
809       XPG
810       YGG
811     YUASA
812       ZEN
813      ZIGA
Name: Symbol, Length: 814, dtype: object

In [29]:
for ticker in tickers:
    df = merged_df.loc[merged_df["Symbol"] == ticker]
    minPE = round(df["PE Ratio"].min(),2)
    avgPE = round(df["PE Ratio"].mean(),2)
    maxPE = round(df["PE Ratio"].max(),2)
    
    minPBV = round(df["PBV Ratio"].min(),2)
    avgPBV = round(df["PBV Ratio"].mean(),2)
    maxPBV = round(df["PBV Ratio"].max(),2)


In [30]:
rf = 0.02
rm = 0.1


In [31]:
dfInfo["K"] = (rf+(rm-rf))*dfInfo.Beta

In [32]:
dfInfo

,Symbol,Recommendation,currentPrice,targetLowPrice,targetMedianPrice,targetMeanPrice,targetHighPrice,Beta,K
0,2S,none,5.40,NaN,NaN,NaN,NaN,0.69292,0.069292
0,3K-BAT,none,78.00,NaN,NaN,NaN,NaN,0.29823,0.029823
0,7UP,none,1.16,NaN,NaN,NaN,NaN,-0.212389,-0.021239
0,A,none,5.00,NaN,NaN,NaN,NaN,0.286283,0.028628
0,A5,none,1.50,NaN,NaN,NaN,NaN,0.156194,0.015619
...,...,...,...,...,...,...,...,...,...
0,XPG,none,2.86,None,None,None,None,1.417805,0.141781
0,YGG,buy,34.00,29,29,29,29,None,NaN
0,YUASA,none,15.70,None,None,None,None,0.881004,0.0881
0,ZEN,none,12.00,14.5,14.5,14.5,14.5,1.423776,0.142378
